In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Masters_Project/Datasets/OASIS3/

/content/drive/MyDrive/Masters_Project/Datasets/OASIS3


In [3]:
!ls

 cifar-10-batches-py	  downloaded_data   t1w_non_ad_jpgs
 cifar-10-python.tar.gz   fa20		    t1w_non_ad_patients
 data			  oasis-scripts    'Usage Instructions.gdoc'


In [4]:
!pip install -r ../../Code/ganomaly/requirements.txt
#!pip install -r ./ganomaly/requirements.txt

In [5]:
!pip install mkl-fft

In [6]:
!pip install Pillow

# imports

In [7]:
import os
import glob
import torch
import numpy as np
import torchvision.datasets as datasets
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms


In [8]:
from PIL import Image
#import cv2
import os.path
import numpy as np
import pickle
from typing import Any, Callable, Optional, Tuple

from torchvision.datasets import VisionDataset
from torchvision.datasets.utils import check_integrity, download_and_extract_archive

In [9]:
print(torch.__version__)

1.2.0


In [10]:
import torchvision
torchvision.__version__

'0.4.0'

# Create Oasis3 data class

In [24]:
transform = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # ERROR Causing https://pytorch.org/docs/stable/torchvision/transforms.html
    ]
)

In [25]:
train_image_number = 140 # Image slice from MRI scans to use for training data

base_folder = '/pyt_oasis3'
non_ad_jpg_folder = './t1w_non_ad_jpgs/' + str(train_image_number) + '/'
ad_jpg_folder = './data/t1w_ad_jpgs/'

class Oasis3_train_normal(VisionDataset):
    def __init__(self,
            root: str,
            train: bool = True, # Train or test dataset
            transform: Optional[Callable] = None,
            target_transform: Optional[Callable] = None) -> None:
        super(Oasis3_train_normal, self).__init__(root, transform=transform, target_transform=target_transform)
        self.train = train # Train or test set
        
        self.data = []
        self.targets = []
        
        for file_path in glob.glob(non_ad_jpg_folder+'**.jpg'):
            with open (file_path, 'rb') as f:
                # image needs to be a PIL image
                img = Image.open(f)
                # Resize all images 176, 256, 3 -> 256, 256, 0
                dsize = (256, 256)
                resized = img.resize(dsize)
                self.data.append(resized)
                self.targets.extend(str(0)) # based on the jpg_folder in for loop
        self.data = np.vstack(self.data).reshape(-1, 256, 256)
        #self.data = self.data.transpose((0, 2, 3, 1))  # convert to HWC
        self.data = self.data.transpose((0, 1, 2))  # convert to HWC

    def __getitem__(self, index: int) -> Tuple[Any, Any]:
        img, target = self.data[index], self.targets[index]
        
        img = Image.fromarray(img)
        
        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return (img, target)

In [26]:
# Issue is i need to make the data transformable inside __getitem__
a=Oasis3_train_normal(base_folder, False, transform=transform)

In [27]:
# OpenCV ---
# Original: (176, 256, 3)
# Re-shaped up: (3, 256, 256)
# After transform: (3, 256, 256)
# PIL ---
# Shape : (256, 256)
a.data[0].shape

(256, 256)

In [28]:
# The class should be index into which invokes the __getitem__ call
print(a[0][0])
print(a[0][1])

RuntimeError: ignored

# PIL image reading

In [ ]:
data = []
with open ('./t1w_non_ad_jpgs/140/OAS30074_ses-d0049_run-01_T1w.jpg', 'rb') as f:
  img = Image.open(f)
  dsize = (256, 256)
  resized = img.resize(dsize)
  data.append(resized)

In [ ]:
data[0].size

# CIFAR data representation testing

In [ ]:
from torchvision.datasets import CIFAR10

In [ ]:
transform = transforms.Compose(
    [
        transforms.Resize(32),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]
)

dataset = {}
dataset['train'] = CIFAR10(root='./', train=True, download=True, transform=transform)

In [ ]:
type(dataset['train'])

In [ ]:
elt = dataset['train'][0]

In [ ]:
type(elt)

In [ ]:
elt[0].shape